<a href="https://colab.research.google.com/github/rein55/Content-Based-Movie-Recommendation-System/blob/main/Content_Based_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dalam mini project ini akan mengambil Content Based Filtering dengan data dari film netflix series. Similarity akan dihitung menggunakan 3 metode (Cosine Similarity, Dice Similarity Coefficient, dan Jaccard Similarity) dan 1 metode tambahan yaitu Euclidean Distance. Terakhir akan dilakukan analisis mengenai hasil content based filtering menggunakan 4 metode tersebut.

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357269 sha256=c24fb83a59ca84a8a6876cf11fe635201514216f068f903dc73b3e0c28ad8813
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
df = pd.read_csv(r'/content/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
df.shape

(8807, 12)

In [ ]:
df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [ ]:
df.isnull().sum()

,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


Notes : Kolom yang diambil untuk diproses lebih lanjut adalah type, title, listed_in, dan description. Kolom selain itu akan dihapus.

In [ ]:
df['listed_in'] = [t.lower() for t in df['listed_in']]
df['description'] = [t.lower() for t in df['description']]

In [ ]:
df.drop(["show_id","director","cast","country","date_added","release_year","rating","duration"],axis=1,inplace=True)
df.head(5)

,type,title,listed_in,description
0,Movie,Dick Johnson Is Dead,documentaries,"as her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t..."
2,TV Show,Ganglands,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...


In [ ]:
df['listed_in'] = df['listed_in'].str.replace(',', ' ', regex=False)
df

,type,title,listed_in,description
0,Movie,Dick Johnson Is Dead,documentaries,"as her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,international tv shows tv dramas tv mysteries,"after crossing paths at a party, a cape town t..."
2,TV Show,Ganglands,crime tv shows international tv shows tv act...,to protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,docuseries reality tv,"feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,international tv shows romantic tv shows tv ...,in a city of coaching centers known to train i...
...,...,...,...,...
8802,Movie,Zodiac,cult movies dramas thrillers,"a political cartoonist, a crime reporter and a..."
8803,TV Show,Zombie Dumb,kids' tv korean tv shows tv comedies,"while living alone in a spooky town, a young g..."
8804,Movie,Zombieland,comedies horror movies,looking to survive in a world taken over by zo...
8805,Movie,Zoom,children & family movies comedies,"dragged from civilian life, a former superhero..."


## COSINE SIMILARITY

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = CountVectorizer()
genre_matrix = vectorizer.fit_transform(df['listed_in'])

In [ ]:
cosine_sim = cosine_similarity(genre_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.80720735, ..., 0.        , 0.        ,
        0.24806947],
       [0.        , 0.80720735, 1.        , ..., 0.        , 0.        ,
        0.10846523],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.57735027,
        0.25819889],
       [0.        , 0.        , 0.        , ..., 0.57735027, 1.        ,
        0.2236068 ],
       [0.        , 0.24806947, 0.10846523, ..., 0.25819889, 0.2236068 ,
        1.        ]])

In [ ]:
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    index=df['title'],  # Use movie titles as row labels
    columns=df['title']  # Use movie titles as column labels
)

cosine_sim_df.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
Blood & Water,0.0,1.000000,0.807207,0.480384,0.832050,0.880705,0.0,0.209657,0.733799,0.196116,...,0.588348,0.277350,0.320256,0.901388,0.277350,0.138675,0.769231,0.000000,0.000,0.248069
Ganglands,0.0,0.807207,1.000000,0.420084,0.848875,0.630126,0.0,0.091670,0.733359,0.000000,...,0.514496,0.121268,0.140028,0.848875,0.121268,0.000000,0.739940,0.000000,0.000,0.108465
Jailbirds New Orleans,0.0,0.480384,0.420084,1.000000,0.433013,0.500000,0.0,0.000000,0.654654,0.000000,...,0.408248,0.000000,0.000000,0.433013,0.000000,0.000000,0.480384,0.000000,0.000,0.000000
Kota Factory,0.0,0.832050,0.848875,0.433013,1.000000,0.649519,0.0,0.094491,0.755929,0.176777,...,0.530330,0.250000,0.144338,0.937500,0.125000,0.000000,0.832050,0.144338,0.125,0.111803


In [ ]:
def recommend_movies(title, similarity_matrix, df=df, n=5):
    # Get the index of the movie that matches the title
    idx = df[df['title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with the given movie
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar movies
    sim_scores = sim_scores[1:n+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top n most similar movies
    return df.iloc[movie_indices]

# Example usage
cosine_recommendations = recommend_movies("Ganglands", cosine_sim)
cosine_recommendations

,type,title,listed_in,description
11,TV Show,Bangkok Breaking,crime tv shows international tv shows tv act...,"struggling to earn a living in bangkok, a man ..."
543,TV Show,Undercover,crime tv shows international tv shows tv act...,undercover agents infiltrate a drug kingpin's ...
734,TV Show,Lupin,crime tv shows international tv shows tv act...,"inspired by the adventures of arsène lupin, ge..."
1223,TV Show,Dealer,crime tv shows international tv shows tv act...,tensions erupt when two filmmakers infiltrate ...
2676,TV Show,Fauda,crime tv shows international tv shows tv act...,a top israeli agent comes out of retirement to...


## Dice Similarity Coefficient

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Transform genre (listed_in) into binary matrix
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(", "))  # Split genre list by commas
genre_matrix = vectorizer.fit_transform(df['listed_in'].fillna("")).toarray()

In [ ]:
# Calculate Dice similarity
# Dice similarity is calculated as 1 - Dice distance
dice_sim = 1 - squareform(pdist(genre_matrix, metric="dice"))  # Dice distance -> similarity

In [ ]:
dice_sim_df = pd.DataFrame(
    dice_sim,
    index=df['title'],  # Use movie titles as row labels
    columns=df['title']  # Use movie titles as column labels
)
dice_sim_df.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blood & Water,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ganglands,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jailbirds New Orleans,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kota Factory,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dice_recommendations = recommend_movies("Ganglands", dice_sim)
dice_recommendations

,type,title,listed_in,description
11,TV Show,Bangkok Breaking,crime tv shows international tv shows tv act...,"struggling to earn a living in bangkok, a man ..."
543,TV Show,Undercover,crime tv shows international tv shows tv act...,undercover agents infiltrate a drug kingpin's ...
734,TV Show,Lupin,crime tv shows international tv shows tv act...,"inspired by the adventures of arsène lupin, ge..."
1223,TV Show,Dealer,crime tv shows international tv shows tv act...,tensions erupt when two filmmakers infiltrate ...
2676,TV Show,Fauda,crime tv shows international tv shows tv act...,a top israeli agent comes out of retirement to...


## JACCARD SIMILARITY

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Transform genre (listed_in) into binary matrix
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(", "))  # Split genre list by commas
genre_matrix = vectorizer.fit_transform(df['listed_in']).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Calculate Jaccard similarity
jaccard_sim = 1 - squareform(pdist(genre_matrix, metric="jaccard"))  # Jaccard distance -> similarity

In [ ]:
jaccard_sim_df = pd.DataFrame(
    jaccard_sim,
    index=df['title'],  # Use movie titles as row labels
    columns=df['title']  # Use movie titles as column labels
)
jaccard_sim_df.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blood & Water,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ganglands,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jailbirds New Orleans,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kota Factory,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
jaccard_recommendations = recommend_movies("Ganglands", jaccard_sim)
jaccard_recommendations

,type,title,listed_in,description
11,TV Show,Bangkok Breaking,crime tv shows international tv shows tv act...,"struggling to earn a living in bangkok, a man ..."
543,TV Show,Undercover,crime tv shows international tv shows tv act...,undercover agents infiltrate a drug kingpin's ...
734,TV Show,Lupin,crime tv shows international tv shows tv act...,"inspired by the adventures of arsène lupin, ge..."
1223,TV Show,Dealer,crime tv shows international tv shows tv act...,tensions erupt when two filmmakers infiltrate ...
2676,TV Show,Fauda,crime tv shows international tv shows tv act...,a top israeli agent comes out of retirement to...


Kesimpulan:

Dari tiga metode similarity yang digunakan dalam tugas content-based filtering ini, yakni Cosine Similarity, Dice Similarity Coefficient, dan Jaccard Similarity, ketiganya menghasilkan rekomendasi lima film teratas yang sama, yaitu Bangkok Breaking, Undercover, Lupin, Dealer, dan Fauda. Kelima film tersebut memiliki genre yang sama dengan Ganglands, sebagaimana terlihat pada kolom listed_in, yang menunjukkan kesamaan genre antara film rekomendasi dan film yang telah ditonton.

Hal ini membuktikan bahwa sistem rekomendasi telah berhasil mengidentifikasi film dengan genre serupa sebagai rekomendasi berdasarkan film yang telah ditonton. Meskipun metode perhitungan similarity pada ketiga pendekatan tersebut berbeda, hasil akhirnya konsisten dalam memberikan rekomendasi yang relevan. Hal ini menegaskan keandalan model dalam memberikan rekomendasi yang sesuai dengan preferensi genre.

## Euclidean Distance

In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = CountVectorizer()
genre_matrix = vectorizer.fit_transform(df['listed_in'])

In [ ]:
# Menghitung jarak Euclidean
euclidean_dist = pairwise_distances(genre_matrix, metric='euclidean')

In [ ]:
# Mengonversi jarak ke dalam DataFrame
euclidean_dist_df = pd.DataFrame(
    euclidean_dist,
    index=df['title'],  # Menggunakan judul film sebagai label baris
    columns=df['title']  # Menggunakan judul film sebagai label kolom
)

euclidean_dist_df.head()

title,Dick Johnson Is Dead,Blood & Water,Ganglands,Jailbirds New Orleans,Kota Factory,Midnight Mass,My Little Pony: A New Generation,Sankofa,The Great British Baking Show,The Starling,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Dick Johnson Is Dead,0.000000,3.741657,4.242641,2.000000,4.123106,3.605551,2.000000,2.828427,2.828427,1.732051,...,1.732051,2.236068,2.000000,4.123106,2.236068,2.236068,3.741657,2.000000,2.236068,2.449490
Blood & Water,3.741657,0.000000,2.449490,3.162278,2.236068,1.732051,4.000000,4.000000,2.449490,3.605551,...,3.000000,3.605551,3.464102,1.732051,3.605551,3.872983,2.449490,4.000000,4.123106,3.741657
Ganglands,4.242641,2.449490,0.000000,3.741657,2.236068,3.316625,4.472136,4.690416,2.828427,4.358899,...,3.605551,4.358899,4.242641,2.236068,4.358899,4.582576,2.828427,4.472136,4.582576,4.472136
Jailbirds New Orleans,2.000000,3.162278,3.741657,0.000000,3.605551,3.000000,2.449490,3.162278,2.000000,2.236068,...,1.732051,2.645751,2.449490,3.605551,2.645751,2.645751,3.162278,2.449490,2.645751,2.828427
Kota Factory,4.123106,2.236068,2.236068,3.605551,0.000000,3.162278,4.358899,4.582576,2.645751,4.000000,...,3.464102,4.000000,4.123106,1.414214,4.242641,4.472136,2.236068,4.123106,4.242641,4.358899


In [ ]:
euclidean_recommendations = recommend_movies("Ganglands", euclidean_dist)
euclidean_recommendations

,type,title,listed_in,description
6423,Movie,Carrie,classic movies cult movies horror movies,an outcast teen with telekinetic ability lashe...
8290,Movie,The Evil Dead,classic movies cult movies horror movies,during a stop at a remote cabin deep within th...
8527,Movie,The Texas Chainsaw Massacre,cult movies horror movies independent movies,when some friends visit a vandalized graveyard...
299,Movie,The Swarm,horror movies independent movies internation...,a single mother breeds locusts as high-protein...
2104,Movie,Dark Forces,horror movies independent movies internation...,"in search of his sister, a renegade criminal s..."


Kesimpulan:

Cosine Similarity, Dice Similarity Coefficient, dan Jaccard Similarity menghasilkan rekomendasi yang sama sedangkan Euclidean Distance menghasilkan rekomendasi yang berbeda dari 3 metode tersebut. Fenomena ini terjadi karena Cosine Similarity, Dice Similarity Coefficient, dan Jaccard Similarity memiliki konsep yang serupa dalam mengukur kemiripan antar item berbasis vektor, sedangkan Euclidean Distance mengukur jarak geometris antar vektor dalam ruang dimensi tinggi. Hasil rekomendasi dari ketiga metode awal lebih sesuai dalam konteks ini dibandingkan dengan Euclidean Distance, karena film rekomendasi yang dihasilkan tetap memiliki genre yang serupa dengan film yang telah ditonton. Sebaliknya, Euclidean Distance menghasilkan rekomendasi yang kurang relevan.

Cosine Similarity, Dice, dan Jaccard cocok digunakan untuk data kategori atau sparse, di mana pola kesamaan lebih penting daripada perbedaan kuantitatif.

Euclidean Distance cocok digunakan ketika ingin mengukur perbedaan absolut antara item dalam ruang multidimensi, terutama pada data dengan variasi nilai numerik.